In [30]:
# Dependencies
import numpy as np
import pandas as pd
import seaborn as sns
import warnings
import matplotlib as mpl
import matplotlib.pyplot as plt
from IPython.display import display, HTML

sns.set("notebook", font_scale=1.05, rc={"figure.figsize": (10, 5)})
sns.set_style("ticks", rc={"figure.facecolor": "none", "axes.facecolor": "none"})
%config InlineBackend.figure_format = 'svg'
warnings.filterwarnings("ignore")
plt.set_loglevel("WARNING")
mpl.rcParams["lines.markersize"] = 8

### Configurations

In [3]:
BASE_DIR = "/home/emerge/daphne/gpudrive/examples/experiments/eval/dataframes/"


# Policy trained with collision_behavior='ignore' on 10 scenes
MODEL_1 = "model_PPO__S_10__01_03_09_18_36_173_000750.csv"

In [4]:
# Load the data
df = pd.read_csv(f"{BASE_DIR}{MODEL_1}", index_col=0)
#df_test = pd.read_csv(f"{BASE_DIR}{RES_TEST}", index_col=0)

In [31]:
df.head()

,scene,goal_achieved,collided,off_road,controlled_agents_in_scene,dataset,train_dataset_size
0,tfrecord-00000-of-01000_1,0.777778,0.0,0.000000,9,train,10
1,tfrecord-00000-of-01000_11,1.000000,0.0,0.000000,3,train,10
2,tfrecord-00000-of-01000_114,1.000000,0.0,0.166667,12,train,10
3,tfrecord-00000-of-01000_118,0.333333,0.0,0.333333,6,train,10
4,tfrecord-00000-of-01000_124,0.857143,0.0,0.142857,7,train,10


### 1. Aggregate performance (todo)

- Aggregate performance train dataset
- Aggregate performance test dataset

In [32]:
df[df['dataset'] == 'train']

,scene,goal_achieved,collided,off_road,controlled_agents_in_scene,dataset,train_dataset_size
0,tfrecord-00000-of-01000_1,0.777778,0.0,0.000000,9,train,10
1,tfrecord-00000-of-01000_11,1.000000,0.0,0.000000,3,train,10
2,tfrecord-00000-of-01000_114,1.000000,0.0,0.166667,12,train,10
3,tfrecord-00000-of-01000_118,0.333333,0.0,0.333333,6,train,10
4,tfrecord-00000-of-01000_124,0.857143,0.0,0.142857,7,train,10
5,tfrecord-00000-of-01000_127,1.000000,0.0,0.000000,3,train,10
6,tfrecord-00000-of-01000_141,0.250000,0.0,0.000000,4,train,10
7,tfrecord-00000-of-01000_15,0.750000,0.0,0.000000,16,train,10
8,tfrecord-00000-of-01000_17,1.000000,0.0,0.000000,2,train,10
9,tfrecord-00000-of-01000_172,0.857143,0.0,0.142857,7,train,10


In [29]:
stats_df = df.groupby('dataset')[['goal_achieved', 'collided', 'off_road']].agg(['mean', 'std'])
stats_df = stats_df * 100  # Convert to percentages
stats_df = stats_df.round(3)  # Round to 3 decimal places

title = "<h3>Aggregate performance (%)</h3>"
display(HTML(title))  # Display the title
display(stats_df)  # Display the DataFrame


goal_achieved         collided         off_road        
                 mean     std     mean     std     mean     std
dataset                                                        
test           99.417   5.077    4.203  13.702    9.778  17.069
train          78.254  27.624    0.000   0.000    7.857  11.449

In [22]:
df[df['dataset'] == 'test']

,scene,goal_achieved,collided,off_road,controlled_agents_in_scene,dataset,train_dataset_size
0,tfrecord-00000-of-00150_10,1.0,0.0,0.333333,3,test,10
1,tfrecord-00000-of-00150_100,1.0,0.0,0.000000,3,test,10
2,tfrecord-00000-of-00150_108,1.0,0.0,0.000000,3,test,10
3,tfrecord-00000-of-00150_11,1.0,0.0,0.000000,5,test,10
4,tfrecord-00000-of-00150_114,1.0,0.0,0.153846,13,test,10
...,...,...,...,...,...,...,...
95,tfrecord-00001-of-00150_193,1.0,0.0,0.200000,5,test,10
96,tfrecord-00001-of-00150_195,1.0,0.0,0.000000,10,test,10
97,tfrecord-00001-of-00150_199,1.0,0.0,0.200000,5,test,10
98,tfrecord-00001-of-00150_20,1.0,0.0,0.125000,8,test,10


### 2. Scaling laws (todo)

### 3. 